# Prompt engineering techniques
1. Zero-shot / Few-shot prompting
2. Chain of Thought (CoT) prompting
3. Self-consistency
4. Tree of Thought (ToT) prompting

## Setup

In [1]:
from dotenv import load_dotenv
import openai

load_dotenv()

True

In [3]:
from openai import OpenAI
client = OpenAI()

def get_completion(prompt, model_name="gpt-3.5-turbo", temperature=0.0):
    messages = [{'role': 'user', 'content': prompt}]
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Zero-shot / One shot prompting

### Zero-shot prompting
Provide prompt without providing examples. Model generates the response based on its knowledge. Ideal for simple tasks. 

In [4]:
text = "A good app overall, but seems to have a few glitches."

prompt = f"""
Classify the text below as positive, mixed or negative. \

'''{text}'''"""

response = get_completion(prompt)
print(response)

Mixed


### Few-shot prompting
Provides examples (questions-answers pairs) in the prompt to steer the model's behavior. Useful for consistent output formatting.

In [5]:
# One-shot prompting
prompt = """
Extract the technical specifications from the text in JSON format.

Specification: honor pad X9, Qualcomm SM6225 Snapdragon 685 chipset, Adreno 610 GPU, 8GB RAM, 7250mAh battery.
Output: 
{
    "tablet": honor pad X9,
    "chipset": Qualcomm SM6225 Snapdragon 685,
    "GPU": Adreno 610,
    "memory": 8GB,
    "battery": 7250mAh
}

Specification: Xiaomi Pad 6, Qualcomm SM8250-AC Snapdragon 870 5G chipset, Adreno 650 GPU, 8GB RAM, 8840mAh battery.
Output:"""

response = get_completion(prompt)
print(response)


{
    "tablet": "Xiaomi Pad 6",
    "chipset": "Qualcomm SM8250-AC Snapdragon 870 5G",
    "GPU": "Adreno 650",
    "memory": "8GB",
    "battery": "8840mAh"
}


In [6]:
# Few-shot prompting
prompt = """
Classify the news title to different topics. \

Title: Arsenal keeper guaranteed league Golden Glove Award.
Topic: Sport

Title: Apple sales fall in nearly all countries.
Topic: Business

Title: Apple has big AI ambitions - at a lower cost than its rivals.
Topic: Technology

Title: Rents for luxury homes in Singapore jump in 1Q amid wider slump.
Topic:
"""

response = get_completion(prompt)
print(response)

Real Estate


## CoT prompting

### Zero shot CoT prompting
**"Let's think step by step"** seems to be a magical word that guide the model's reasoning, thereby increasing the likelihood of an accurate answer. Zero-shot-CoT may not be as effective as CoT prompting, when the reasoning tasks are more complex. This technique was first introduced in [Kojima et al.(2022)](https://arxiv.org/abs/2205.11916).

In [17]:
question = """
Bob has 4 shuttlecocks. He buys 2 more tubes of shuttlecocks. Each tube has 12 shuttlecocks. How many shuttlecocks does he has now?"""

response = get_completion(question)
print(response)

Bob initially has 4 shuttlecocks. 
He buys 2 tubes of shuttlecocks, each containing 12 shuttlecocks. 
So, he buys a total of 2 * 12 = 24 shuttlecocks.
Adding the initial 4 shuttlecocks, Bob now has a total of 4 + 24 = 28 shuttlecocks. 

Therefore, Bob now has 28 shuttlecocks.


In [18]:
question = """
Bob has 4 shuttlecocks. He buys 2 more tubes of shuttlecocks. Each tube has 12 shuttlecocks. How many shuttlecocks does he has now? \
Let's think step by step."""

response = get_completion(question)
print(response)

Step 1: Bob starts with 4 shuttlecocks.
Step 2: He buys 2 tubes of shuttlecocks, each containing 12 shuttlecocks.
Step 3: The total number of shuttlecocks in the tubes is 2 tubes * 12 shuttlecocks per tube = 24 shuttlecocks.
Step 4: Adding the shuttlecocks Bob already had, the total number of shuttlecocks he has now is 4 shuttlecocks + 24 shuttlecocks = 28 shuttlecocks.

Therefore, Bob now has 28 shuttlecocks.


In [11]:
question = """
There is a group of 10 people who are ordering pizza. If each person gets 2 slices and each pizza has 4 slices, \
how many pizzas should they order?"""

response = get_completion(question)
print(response)

They should order 5 pizzas. 

10 people x 2 slices per person = 20 slices needed
20 slices / 4 slices per pizza = 5 pizzas needed


In [12]:
question = """
Was Aristotle alive when laptop was invented? Yes or No."""

response = get_completion(question)
print(response)

No. Aristotle lived in ancient Greece from 384-322 BC, while the first laptop was invented in 1981.


In [13]:
question = """
Bob buys 2 more tubes of shuttlecocks. Each tube has 12 shuttlecocks, whereby each shuttlecocks cost RM7. \
If he received 5% discounts for each tube he buy, how much in total Bob has to pay?"""

response = get_completion(question)
print(response)

First, let's calculate the total cost of the shuttlecocks without any discounts:

Cost per shuttlecock = RM7
Shuttlecocks per tube = 12
Total cost per tube = RM7 * 12 = RM84

Bob buys 2 more tubes, so the total cost without any discounts would be:
Total cost for 2 tubes = RM84 * 2 = RM168

Now, let's calculate the total cost with the 5% discount for each tube:

Discount per tube = 5% of RM84 = 0.05 * RM84 = RM4.20
Total discount for 2 tubes = RM4.20 * 2 = RM8.40

Total cost after discount = Total cost without discount - Total discount
Total cost after discount = RM168 - RM8.40 = RM159.60

Therefore, Bob has to pay RM159.60 in total after receiving the 5% discount for each tube he buys.


In [14]:
question = """
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. \
I then went and bought 5 more apples and ate 1. How many apples did I remain with?"""

response = get_completion(question)
print(response)

You started with 10 apples, gave away 4, bought 5 more, and ate 1. 

So, you remained with 10 - 4 + 5 - 1 = 10 apples. 

You remained with 10 apples.


### Limitations of LLMs when it comes to complex reasoning problem even for zero-shot CoT prompting.

In [8]:
question = """
There is a three-digit number. The second digit is four times as big as the third digit, \
while the first digit is three less than the second digit. What is the number?"""

response = get_completion(question)
print(response)

Let's denote the three digits as A, B, and C.

According to the given conditions:
B = 4C
A = B - 3

Substitute the value of B from the first equation into the second equation:
A = 4C - 3

Since A, B, and C are digits, they must be between 0 and 9. From the first equation, we know that B = 4C, so C can only be 1 or 2. If C = 1, then B = 4 and A = 1, which doesn't satisfy the condition that A is three less than B. Therefore, C must be 2.

If C = 2, then B = 4(2) = 8 and A = 8 - 3 = 5.

Therefore, the three-digit number is 582.


In [9]:
question = """
There is a three-digit number. The second digit is four times as big as the third digit, \
while the first digit is three less than the second digit. What is the number? \
Let's think step by step."""

response = get_completion(question)
print(response)

Step 1: Let's represent the three-digit number as ABC, where A is the first digit, B is the second digit, and C is the third digit.

Step 2: We know that the second digit is four times as big as the third digit, so we can write this as B = 4C.

Step 3: We also know that the first digit is three less than the second digit, so we can write this as A = B - 3.

Step 4: Substituting the value of B from step 2 into step 3, we get A = 4C - 3.

Step 5: Since the number is a three-digit number, A cannot be 0. Therefore, A must be 1, 2, 3, 4, 5, 6, 7, 8, or 9.

Step 6: Let's try each value of A and see if we can find a valid solution.

If A = 1, then B = 4C + 3. Since B is a digit, the only possible value for C is 1. However, this would make B = 7, which is not a valid digit.

If A = 2, then B = 4C + 3. The only possible value for C is 1, which gives us B = 7. Therefore, the number is 217.

Therefore, the three-digit number is 217.


### Few shot CoT prompting

In [19]:
question = """
Which is a faster way to get to work?
Option 1: Take a 15 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take a 5 minute bus, then an hour train, and finally a 120 seconds walk.
Reasoning: \
Option 1: 15 + (0.5*60) + 10 = 55 minutes
Option 2: 5 + 60 + (120 / 60) = 67 minutes
So, option 1 is faster than option 2.

Option 1: Take an 10 minutes bus, then an 40 minute bus, and finally a 10 minute train.
Option 2: Take a 9 minutes train, then a 45 minute bike ride, and finally a 10 minute bus.
Reasoning: 
"""

response = get_completion(question)
print(response)

Option 1: 10 + 40 + 10 = 60 minutes
Option 2: 9 + 45 + 10 = 64 minutes
So, option 1 is faster than option 2.


In [16]:
question = """
Which is a faster way to get to work?
Option 1: Take a 15 minute bus, then a half hour train, and finally a 10 minute bike ride.
Option 2: Take a 5 minute bus, then an hour train, and finally a 120 seconds walk."""

response = get_completion(question)
print(response)

Option 1 is the faster way to get to work. It involves a total travel time of 55 minutes (15 minutes bus + 30 minutes train + 10 minute bike ride), while Option 2 involves a total travel time of 65 minutes (5 minutes bus + 60 minutes train + 120 seconds walk).


## Self-consistency
Self-consistency prompting is a technique that generates multiple CoT by prompting the model several times to obtain different outputs. The final outputs is then obtained by a majority vote, where the most commonly occurring output or response is selected as final result.

In [20]:
def chain_of_thought_prompts(prompt, model='gpt-3.5-turbo', iterations=3, temperature=0.0):
    current_prompt = prompt + " Let's think step by step."
    messages = [{"role": "user", "content": current_prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        n=iterations,
        temperature=temperature
    )
    responses = [choice.message.content for choice in response.choices if choice.message.role == 'assistant']
    return responses

In [22]:
prompt = """
When I was 10 my sister was half my age. Now I'm 32 how old is my sister?"""

responses = chain_of_thought_prompts(prompt, temperature=0.3)
print(responses)

['When you were 10, your sister was half your age, which means she was 5 years old at that time.\n\nNow you are 32, which means there is a 22-year age gap between you and your sister.\n\nSo, if your sister was 5 when you were 10, she would be 5 + 22 = 27 years old now.', 'When you were 10, your sister was half your age, so she was 5 years old. \n\nNow that you are 32, 22 years have passed since you were 10. \n\nTherefore, your sister is now 5 + 22 = 27 years old.', "When you were 10, your sister was half your age, which means she was 5 years old at that time.\n\nNow you are 32 years old, so if we add the age difference between you and your sister (32-10 = 22), we can determine your sister's current age.\n\nTherefore, your sister is now 27 years old."]


In [25]:
new_prompt = ""
for i, s in enumerate(responses):
    # remove newline within a single output
    s = s.replace("\n", " ")
    new_prompt += f"Expert {i+1}: " + s + " \n"

new_prompt += "Based on majority voting, what will be the age of my sister when I am 32?"
print(new_prompt)

Expert 1: When you were 10, your sister was half your age, which means she was 5 years old at that time.  Now you are 32, which means there is a 22-year age gap between you and your sister.  So, if your sister was 5 when you were 10, she would be 5 + 22 = 27 years old now. 
Expert 2: When you were 10, your sister was half your age, so she was 5 years old.   Now that you are 32, 22 years have passed since you were 10.   Therefore, your sister is now 5 + 22 = 27 years old. 
Expert 3: When you were 10, your sister was half your age, which means she was 5 years old at that time.  Now you are 32 years old, so if we add the age difference between you and your sister (32-10 = 22), we can determine your sister's current age.  Therefore, your sister is now 27 years old. 
Based on majority voting, what will be the age of my sister when I am 32?


In [26]:
response = get_completion(new_prompt)
print(response)

The age of your sister when you are 32 will be 27 years old, according to the majority voting of the experts.


## Tree-of-thought (ToT) prompting
ToT maintains a tree of thoughts, where thoughts represent coherent language sequences that serve as intermediate steps toward solving a problem. This approach enables an LM to self-evaluate the progress through intermediate thoughts made towards solving a problem through a deliberate reasoning process. 

[Hulbert (2023)](https://github.com/dave1010/tree-of-thought-prompting/blob/main/tree-of-thought-prompts.txt) has proposed Tree-of-Thought Prompting, which applies the main concept from ToT frameworks as a simple prompting technique, getting the LLM to evaluate intermediate thoughts in a single prompt.

In [27]:
question = "28, 29, 27, 28, 26, ..."

prompt = f"""
Identify and behave as three different experts that are appropriate to answering this question. \
All experts will write down the step and their thinking about the step, then share it with the group. \
Then, all experts will go on to the next step, etc. \
At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, \
and 5 meaning it is highly likely.
If any expert is judged to be wrong at any point then they leave. \
After all experts have provided their analysis, you then analyze all 3 analyses and provide either \
the consensus solution or your best guess solution.

What is the number that comes next for the number series below? \
Number series: '''{question}'''"""

response = get_completion(prompt, temperature=0.5)
print(response)

Expert 1 (Mathematician):
Step: Identify the pattern in the number series.
Thinking: The pattern seems to be alternating between adding 1 and subtracting 2.
Score: 4

Expert 2 (Statistician):
Step: Analyze the frequency of the numbers in the series.
Thinking: The numbers 28, 27, and 26 appear once each, while 29 appears twice. Following this pattern, the next number should be 27.
Score: 4

Expert 3 (Data Scientist):
Step: Use a mathematical formula to predict the next number.
Thinking: The formula for this series seems to be (n-1) + (-1)^n, where n is the position in the series. Plugging in n=6, we get (6-1) + (-1)^6 = 5 - 1 = 4. Therefore, the next number should be 4.
Score: 3

Consensus/Best guess solution: Based on the analysis provided by the mathematician and statistician, it seems more likely that the next number in the series is 27. Therefore, the consensus solution is 27.
